In [ ]:
%pip uninstall azureml-dataprep-rslex -y
%pip install -U azureml-dataprep-rslex --extra-index-url=https://dataprepdownloads.azureedge.net/pypi/test-M3ME5B1GMEM3SW0W/62172131/

Found existing installation: azureml-dataprep-rslex 2.5.3
Uninstalling azureml-dataprep-rslex-2.5.3:
  Successfully uninstalled azureml-dataprep-rslex-2.5.3
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://dataprepdownloads.azureedge.net/pypi/test-M3ME5B1GMEM3SW0W/62172131/
  Using cached azureml_dataprep_rslex-2.5.3-cp38-cp38-manylinux1_x86_64.whl (15.5 MB)
ERROR: azureml-dataprep 3.0.0 has requirement azureml-dataprep-rslex~=2.4.0dev0, but you'll have azureml-dataprep-rslex 2.5.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from azureml.core import Workspace, Dataset
from azureml.data import DataType

ws = Workspace.from_config()
dstore = ws.datastores['testfiles']
labeled_dset = Dataset.Tabular.from_json_lines_files(dstore.path('mnist_labeled_ds/*.jsonl'), set_column_types={'image_url': DataType.to_stream(ws)})
labeled_dset.to_pandas_dataframe()

,image_url,label,label_confidence,image_height,image_width
0,testfiles/numbers/4/2323.png,4,1.0,28,28
1,testfiles/numbers/4/25662.png,4,1.0,28,28
2,testfiles/numbers/3/24196.png,3,1.0,28,28
3,testfiles/numbers/9/21192.png,9,1.0,28,28
4,testfiles/numbers/9/21267.png,9,1.0,28,28
...,...,...,...,...,...
8850,testfiles/numbers/7/55384.png,7,1.0,28,28
8851,testfiles/numbers/4/31860.png,4,1.0,28,28
8852,testfiles/numbers/4/59708.png,4,1.0,28,28
8853,testfiles/numbers/6/59509.png,6,1.0,28,28


In [ ]:
from torch.utils.data import Dataset

class AzureMLDataset(Dataset):
    def __init__(self, dataset, buffering_options=None, data_transforms=None, label_transforms=None):
        self._dataframe = dataset.to_pandas_dataframe()
        self._buffering_options = buffering_options
        self._data_transforms = data_transforms
        self._label_transforms = label_transforms

    def __len__(self):
        return len(self._dataframe)

    def __getitem__(self, idx):
        from PIL import Image

        image_stream = self._dataframe.iloc[idx, 0]
        with image_stream.open(self._buffering_options) as f:
            image = Image.open(f)
            if self._data_transforms:
                image = self._data_transforms(image)

        label = self._dataframe.iloc[idx, 1]
        if self._label_transforms:
            label = self._label_transforms(label)

        return image, label

In [ ]:
smol_dataset = labeled_dset.take_sample(0.1)
smol_train, smol_test = smol_dataset.random_split(0.9)

import mlflow
import torch
from train import train, test, get_data_loaders, Net
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

mlflow.autolog()

torch.manual_seed(1)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_loader, test_loader = get_data_loaders(AzureMLDataset(smol_train,data_transforms=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),label_transforms=int), 
                                             AzureMLDataset(smol_test,data_transforms=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),label_transforms=int))

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

args = {'log_interval': 100}
train(args, model, device, train_loader, optimizer, 0)
test(args, model, device, test_loader)

2022/05/04 23:24:24 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2022/05/04 23:24:24 INFO mlflow.pyspark.ml: No SparkSession detected. Autologging will log pyspark.ml models contained in the default allowlist. To specify a custom allowlist, initialize a SparkSession prior to calling mlflow.pyspark.ml.autolog() and specify the path to your allowlist file via the spark.mlflow.pysparkml.autolog.logModelAllowlistFile conf.
2022/05/04 23:24:24 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


TypeError: 'NoneType' object cannot be interpreted as an integer